In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Dec 24 12:22:36 2022

@author: eniseranabeklen
"""


from selenium import webdriver
from shutil import which
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException
from selenium.webdriver.common.by import By
import pandas as pd
import time

# fetch_jobs functions takes keyword, num_pages, and location as arguments. The function opens Glassdoor on Google Chrome and uses its arguments to search jobs.

def fetch_jobs(keyword, num_pages, location):
    options = Options()
    options.add_argument("window-size=1920,1080")
    #Enter your chromedriver.exe path below
    chrome_path = r"C/Users/eniseranabeklen/Documents/GitHub/ds_salary_proj/chromedriver.exe"
    driver = webdriver.Chrome(executable_path=chrome_path, options=options)
    driver.get("https://www.glassdoor.com/Job/index.htm")
    search_input = driver.find_element(By.ID, "KeywordSearch")
    location_input = driver.find_element(By.ID, "LocationSearch")
    location_input.clear()
    search_input.send_keys(keyword)
    location_input.send_keys(location)
    driver.find_element(By.ID, "HeroSearchButton").click()
    time.sleep(2)
    
# we are creating lists to save the scraped data.     
    company_name = []
    job_title = []
    salary_est = []
    location = []
    job_description = []
    salary_estimate = []
    company_size = []
    company_type = []
    company_sector = []
    company_industry = []
    company_founded = []
    company_revenue = []
    
    
    #Set current page to 1
    current_page = 1     
    time.sleep(3)
    
    while current_page <= num_pages:   
        
        done = False
        count = 0 
        while not done:
            #job_cards = driver.find_elements(By.XPATH, "//article[@id='MainCol']//ul/li[@data-adv-type='GENERAL']")
            
            job_cards = driver.find_elements(By.XPATH, "//li[@data-adv-type='GENERAL']")
            
            for card in job_cards:
                card.click()
                time.sleep(3)
                count += 1
                #Closes the signup prompt
                try:
                    driver.find_element(By.XPATH,".//span[@class='SVGInline modal_closeIcon']").click()
                    time.sleep(2)
                except NoSuchElementException:
                    time.sleep(2)
                    pass

                #Expands the Description section by clicking on Show More
                try:
                    driver.find_element(By.XPATH,"//*[@id='JobDescriptionContainer']/div[2]").click()
                    time.sleep(1)
                except NoSuchElementException:
                    card.click()
                    print(str(current_page) + '#ERROR: no such element')
                    driver.refresh()
                    pass
                except ElementNotInteractableException:
                    card.click()
                    driver.implicitly_wait(30)
                    driver.refresh()
                    print(str(current_page) + '#ERROR: not interactable')
                    pass

                #Scrape 
                try:
                    company_name.append(driver.find_element(By.XPATH,"//*[@id='JDCol']/div/article/div/div[1]/div/div/div/div/div[1]/div[1]/div").text)
                except:
                    company_name.append("#N/A")
                    pass

                try:
                    job_title.append(driver.find_element(By.XPATH,'//*[@id="JDCol"]/div/article/div/div[1]/div/div/div/div/div[1]/div[2]').text)
                except:
                    job_title.append("#N/A")
                    pass

                try:
                    location.append(driver.find_element(By.XPATH,'//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[3]').text)
                except:
                    location.append("#N/A")
                    pass

                try:
                    job_description.append(driver.find_element(By.XPATH,"//div[@id='JobDescriptionContainer']").text)
                except:
                    job_description.append("#N/A")
                    pass
                
                #we need to iterate through job listings to extract salary estimate data
                xpath = '//*[@id="MainCol"]/div[1]/ul/li[' + str(count) + ']/div[2]/div[3]/div[1]/span'
                try:
                    salary_estimate.append(driver.find_element(By.XPATH, xpath).text)
                except:
                    salary_estimate.append("#N/A")
                    pass
                
                try:
                    company_size.append(driver.find_element(By.XPATH,"//div[@id='CompanyContainer']//span[text()='Size']//following-sibling::*").text)
                except:
                    company_size.append("#N/A")
                    pass
                
                try:
                    company_type.append(driver.find_element(By.XPATH,"//div[@id='CompanyContainer']//span[text()='Type']//following-sibling::*").text)
                except:
                    company_type.append("#N/A")
                    pass
                    
                try:
                    company_sector.append(driver.find_element(By.XPATH,"//div[@id='CompanyContainer']//span[text()='Sector']//following-sibling::*").text)
                except:
                    company_sector.append("#N/A")
                    pass
                    
                try:
                    company_industry.append(driver.find_element(By.XPATH,"//div[@id='CompanyContainer']//span[text()='Industry']//following-sibling::*").text)
                except:
                    company_industry.append("#N/A")
                    pass
                     
                try:
                    company_founded.append(driver.find_element(By.XPATH,"//div[@id='CompanyContainer']//span[text()='Founded']//following-sibling::*").text)
                except:
                    company_founded.append("#N/A")
                    pass
                    
                try:
                    company_revenue.append(driver.find_element(By.XPATH,"//div[@id='CompanyContainer']//span[text()='Revenue']//following-sibling::*").text)
                except:
                    company_revenue.append("#N/A")
                    pass

                    
                done = True
                
       # Moves to the next page         
        if done:
            print(str(current_page) + ' ' + 'out of' +' '+ str(num_pages) + ' ' + 'pages done')
            driver.find_element(By.XPATH,"//span[@alt='next-icon']").click()   
            current_page = current_page + 1
            time.sleep(4)


    driver.close()
    df = pd.DataFrame({'company': company_name, 
        'job title': job_title,
        'location': location,
        'job description': job_description,
        'salary estimate': salary_estimate,
        'company_size': company_size,
        'company_type': company_type,
        'company_sector': company_sector,
        'company_industry' : company_industry, 
        'company_founded' : company_founded, 
        'company_revenue': company_revenue})
    

    df.to_csv(keyword + '.csv')

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Dec 24 12:22:36 2022

@author: eniseranabeklen
"""


from selenium import webdriver
from shutil import which
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException
from selenium.webdriver.common.by import By
import pandas as pd
import time

# fetch_jobs functions takes keyword, num_pages, and location as arguments. The function opens Glassdoor on Google Chrome and uses its arguments to search jobs.

def fetch_jobs(keyword, num_pages, location):
    options = Options()
    options.add_argument("window-size=1920,1080")
    driver = webdriver.Chrome(options=options)  # auto uses ChromeDriver if installed
    driver.get("https://www.glassdoor.com/Job/index.html")
    
    search_input = driver.find_element(By.ID, "KeywordSearch")
    location_input = driver.find_element(By.ID, "LocationSearch")
    location_input.clear()
    search_input.send_keys(keyword)
    location_input.send_keys(location)
    driver.find_element(By.ID, "HeroSearchButton").click()
    time.sleep(3)

    # Initialize lists
    company_name, job_title, location_list, job_description, salary_estimate = [], [], [], [], []
    company_size, company_type, company_sector, company_industry, company_founded, company_revenue = [], [], [], [], [], []
    
    current_page = 1     
    while current_page <= num_pages:   
        job_cards = driver.find_elements(By.XPATH, "//li[@data-adv-type='GENERAL']")
        
        for idx, card in enumerate(job_cards, start=1):
            try:
                card.click()
                time.sleep(2)

                # Close popups if they appear
                try:
                    driver.find_element(By.XPATH,".//span[@class='SVGInline modal_closeIcon']").click()
                    time.sleep(1)
                except NoSuchElementException:
                    pass

                # Scraping
                try:
                    company_name.append(driver.find_element(By.XPATH,"//*[@id='JDCol']//div[1]//div[1]//div[1]").text)
                except: company_name.append("#N/A")

                try:
                    job_title.append(driver.find_element(By.XPATH,'//*[@id="JDCol"]//div[2]').text)
                except: job_title.append("#N/A")

                try:
                    location_list.append(driver.find_element(By.XPATH,'//*[@id="JDCol"]//div[3]').text)
                except: location_list.append("#N/A")

                try:
                    job_description.append(driver.find_element(By.ID,"JobDescriptionContainer").text)
                except: job_description.append("#N/A")

                try:
                    salary_estimate.append(driver.find_element(By.XPATH, f'//*[@id="MainCol"]/div[1]/ul/li[{idx}]/div[2]/div[3]/div[1]/span').text)
                except: salary_estimate.append("#N/A")

                # Company info
                try: company_size.append(driver.find_element(By.XPATH,"//div[@id='CompanyContainer']//span[text()='Size']//following-sibling::*").text)
                except: company_size.append("#N/A")
                try: company_type.append(driver.find_element(By.XPATH,"//div[@id='CompanyContainer']//span[text()='Type']//following-sibling::*").text)
                except: company_type.append("#N/A")
                try: company_sector.append(driver.find_element(By.XPATH,"//div[@id='CompanyContainer']//span[text()='Sector']//following-sibling::*").text)
                except: company_sector.append("#N/A")
                try: company_industry.append(driver.find_element(By.XPATH,"//div[@id='CompanyContainer']//span[text()='Industry']//following-sibling::*").text)
                except: company_industry.append("#N/A")
                try: company_founded.append(driver.find_element(By.XPATH,"//div[@id='CompanyContainer']//span[text()='Founded']//following-sibling::*").text)
                except: company_founded.append("#N/A")
                try: company_revenue.append(driver.find_element(By.XPATH,"//div[@id='CompanyContainer']//span[text()='Revenue']//following-sibling::*").text)
                except: company_revenue.append("#N/A")

                # Print each scraped job
                print(f"✅ {job_title[-1]} at {company_name[-1]} ({location_list[-1]})")

            except Exception as e:
                print("Error scraping job:", e)

        # Go to next page
        try:
            next_button = driver.find_element(By.XPATH, "//button[@aria-label='Next']")
            next_button.click()
            current_page += 1
            time.sleep(4)
        except NoSuchElementException:
            break

    driver.quit()

    df = pd.DataFrame({
        'company': company_name, 
        'job title': job_title,
        'location': location_list,
        'job description': job_description,
        'salary estimate': salary_estimate,
        'company size': company_size,
        'company type': company_type,
        'company sector': company_sector,
        'company industry': company_industry,
        'company founded': company_founded,
        'company revenue': company_revenue
    })

    print(df.head())
    df.to_csv(f"{keyword}.csv", index=False)
    return df

In [11]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [14]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''

    # Initializing the webdriver
    options = webdriver.ChromeOptions()

    # Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    # options.add_argument('headless')

    # FIX: Use Service instead of executable_path
    service = Service(path)
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(1120, 1000)


    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_css_selector('[alt="Close"]').click() #clicking to the X.
            print(' x out worked')
        except NoSuchElementException:
            print(' x out failed')
            pass


        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
            try:
                job_button.click()  #You might 
                time.sleep(1)
                collected_successfully = False
            except:
                continue
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."

            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1


            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs


        #Clicking on the "next page" button
        try:
            page = driver.find_element_by_xpath('.//div[@class="tbl fill padHorz margVert"]').text
            page = page.split()
            if page[1]==page[3]:
                break
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs).reset_index()  #This line converts the dictionary object into a pandas DataFrame.

In [15]:
path = "chromedriver"  # make sure chromedriver is in PATH or give full path e.g. "C:/path/to/chromedriver.exe"
df = get_jobs('data scientist', 1500, False, path, 4)
df.to_csv('Uncleaned_DS_jobs.csv', index=False)

NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [17]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd


def get_jobs(keyword, num_jobs, verbose, slp_time):
    """Gathers jobs as a dataframe, scraped from Glassdoor"""

    # Initialize Chrome driver with webdriver_manager
    options = webdriver.ChromeOptions()
    # options.add_argument("headless")  # Uncomment if you want headless mode
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.set_window_size(1120, 1000)

    url = f"https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword={keyword}&sc.keyword={keyword}&locT=&locId=&jobType="
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:
        time.sleep(slp_time)

        # Close sign-up/login popup if it appears
        try:
            driver.find_element("css selector", '[alt="Close"]').click()
            print("Popup closed")
        except NoSuchElementException:
            pass


        time.sleep(0.1)

        try:
            driver.find_element("css selector", '[alt="Close"]').click()
            print("Popup closed")
        except NoSuchElementException:
            pass

        # Loop through job listings
        job_buttons = driver.find_elements("class name", "jl")  # job listing buttons
        for job_button in job_buttons:
            print(f"Progress: {len(jobs)}/{num_jobs}")
            if len(jobs) >= num_jobs:
                break

            try:
                job_button.click()
                time.sleep(1)
                collected_successfully = False
            except:
                continue

            while not collected_successfully:
                try:
                    company_name = driver.find_element("xpath", './/div[@class="employerName"]').text
                    location = driver.find_element("xpath", './/div[@class="location"]').text
                    job_title = driver.find_element("xpath", './/div[contains(@class, "title")]').text
                    job_description = driver.find_element("xpath", './/div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element("xpath", './/span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1

            try:
                rating = driver.find_element("xpath", './/span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1

            if verbose:
                print(f"Job Title: {job_title}")
                print(f"Salary Estimate: {salary_estimate}")
                print(f"Job Description: {job_description[:500]}")
                print(f"Rating: {rating}")
                print(f"Company Name: {company_name}")
                print(f"Location: {location}")

            # Go to Company tab
            try:
                driver.find_element("xpath", './/div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    headquarters = driver.find_element("xpath", './/div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element("xpath", './/div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element("xpath", './/div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element("xpath", './/div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element("xpath", './/div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element("xpath", './/div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element("xpath", './/div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element("xpath", './/div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:
                headquarters = size = founded = type_of_ownership = industry = sector = revenue = competitors = -1

            if verbose:
                print(f"Headquarters: {headquarters}")
                print(f"Size: {size}")
                print(f"Founded: {founded}")
                print(f"Type of Ownership: {type_of_ownership}")
                print(f"Industry: {industry}")
                print(f"Sector: {sector}")
                print(f"Revenue: {revenue}")
                print(f"Competitors: {competitors}")
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({
                "Job Title": job_title,
                "Salary Estimate": salary_estimate,
                "Job Description": job_description,
                "Rating": rating,
                "Company Name": company_name,
                "Location": location,
                "Headquarters": headquarters,
                "Size": size,
                "Founded": founded,
                "Type of ownership": type_of_ownership,
                "Industry": industry,
                "Sector": sector,
                "Revenue": revenue,
                "Competitors": competitors
            })

        # Next page
        try:
            page = driver.find_element("xpath", './/div[@class="tbl fill padHorz margVert"]').text
            page = page.split()
            if page[1] == page[3]:
                break
            driver.find_element("xpath", './/li[@class="next"]//a').click()
        except NoSuchElementException:
            print(f"Scraping terminated early. Needed {num_jobs}, got {len(jobs)}.")
            break

    driver.quit()
    return pd.DataFrame(jobs).reset_index()


# ----------------- Run scraper -----------------
df = get_jobs("data scientist", 50, False, 4)  # Try small number first (50)
df.to_csv("Uncleaned_DS_jobs.csv", index=False)
print("Scraping completed and saved to Uncleaned_DS_jobs.csv")

Scraping terminated early. Needed 50, got 0.
Scraping completed and saved to Uncleaned_DS_jobs.csv


In [18]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd


def get_jobs(keyword, num_jobs, verbose, slp_time):
    """Gathers jobs as a dataframe, scraped from Glassdoor"""

    # Initialize Chrome driver with webdriver_manager
    options = webdriver.ChromeOptions()
    # options.add_argument("headless")  # Uncomment if you want headless mode
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.set_window_size(1120, 1000)

    # Open Glassdoor search page
    url = f"https://www.glassdoor.com/Job/jobs.htm?sc.keyword={keyword}"
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:
        time.sleep(slp_time)

        # Close signup/login popup if it appears
        try:
            driver.find_element("css selector", '[alt="Close"]').click()
            print("Popup closed")
        except NoSuchElementException:
            pass

        # Find job cards
        job_cards = driver.find_elements("css selector", "li.react-job-listing")

        for job_card in job_cards:
            print(f"Progress: {len(jobs)}/{num_jobs}")
            if len(jobs) >= num_jobs:
                break

            try:
                job_card.click()
                time.sleep(2)  # wait for details to load
            except:
                continue

            # Extract job info
            try:
                job_title = driver.find_element("css selector", "div[data-test='jobTitle']").text
            except NoSuchElementException:
                job_title = -1

            try:
                company_name = driver.find_element("css selector", "div[data-test='employerName']").text
            except NoSuchElementException:
                company_name = -1

            try:
                location = driver.find_element("css selector", "div[data-test='location']").text
            except NoSuchElementException:
                location = -1

            try:
                job_description = driver.find_element("css selector", "div.jobDescriptionContent").text
            except NoSuchElementException:
                job_description = -1

            try:
                salary_estimate = driver.find_element("css selector", "span[data-test='detailSalary']").text
            except NoSuchElementException:
                salary_estimate = -1

            try:
                rating = driver.find_element("css selector", "span[data-test='detailRating']").text
            except NoSuchElementException:
                rating = -1

            if verbose:
                print(f"Job Title: {job_title}")
                print(f"Company: {company_name}")
                print(f"Location: {location}")
                print(f"Salary: {salary_estimate}")
                print(f"Rating: {rating}")
                print(f"Description (first 200 chars): {job_description[:200]}")
                print("-" * 50)

            jobs.append({
                "Job Title": job_title,
                "Company Name": company_name,
                "Location": location,
                "Salary Estimate": salary_estimate,
                "Rating": rating,
                "Job Description": job_description,
            })

        # Try to go to next page
        try:
            next_button = driver.find_element("css selector", "button[data-test='pagination-next']")
            if not next_button.is_enabled():
                print("No more pages.")
                break
            next_button.click()
        except NoSuchElementException:
            print(f"Scraping terminated early. Needed {num_jobs}, got {len(jobs)}.")
            break

    driver.quit()
    return pd.DataFrame(jobs).reset_index(drop=True)


# ----------------- Run scraper -----------------
df = get_jobs("data scientist", 50, True, 4)  # start with 50 jobs
df.to_csv("Uncleaned_DS_jobs.csv", index=False)
print("✅ Scraping completed and saved to Uncleaned_DS_jobs.csv")

Scraping terminated early. Needed 50, got 0.
✅ Scraping completed and saved to Uncleaned_DS_jobs.csv


In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time


def get_jobs(keyword, num_jobs, verbose, slp_time):
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.set_window_size(1120, 1000)

    url = f"https://www.glassdoor.com/Job/jobs.htm?sc.keyword={keyword}"
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:
        try:
            job_cards = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[data-test='job-card-wrapper']"))
            )
        except TimeoutException:
            print("⚠️ No job cards found. Possible block or structure change.")
            break

        for job_card in job_cards:
            if len(jobs) >= num_jobs:
                break

            try:
                job_card.click()
                time.sleep(2)
            except:
                continue

            # Safe finder
            def safe_find(selector, by=By.CSS_SELECTOR):
                try:
                    return driver.find_element(by, selector).text
                except NoSuchElementException:
                    return -1

            job_title = safe_find("a[data-test='job-title']")
            company_name = safe_find("span.EmployerProfile_compactEmployerName__9MGcV")
            location = safe_find("div[data-test='emp-location']")
            salary_estimate = safe_find("span[id^='job-salary']")
            rating = safe_find("span.rating-single-star_RatingText__5fdjN")
            job_description = safe_find("div.jobDescriptionContent")

            jobs.append({
                "Job Title": job_title,
                "Company Name": company_name,
                "Location": location,
                "Salary Estimate": salary_estimate,
                "Rating": rating,
                "Job Description": job_description,
            })

        # Go to next page
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "button[data-test='pagination-next']")
            if next_button.is_enabled():
                next_button.click()
                time.sleep(slp_time)
            else:
                break
        except NoSuchElementException:
            break

    driver.quit()
    return pd.DataFrame(jobs)


# ---------------- Run scraper ----------------
df = get_jobs("data scientist", 20, True, 4)
df.to_csv("Uncleaned_DS_jobs.csv", index=False)
print(f"✅ Got {len(df)} jobs, saved to Uncleaned_DS_jobs.csv")

✅ Got 20 jobs, saved to Uncleaned_DS_jobs.csv


In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time


def get_jobs(keyword, num_jobs, verbose, slp_time):
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.set_window_size(1120, 1000)

    url = f"https://www.glassdoor.com/Job/jobs.htm?sc.keyword={keyword}"
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:
        try:
            job_cards = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.JobCard_jobCardWrapper__vX29z"))
                )

        except TimeoutException:
            print("⚠️ No job cards found. Possible block or structure change.")
            break

        for job_card in job_cards:
            if len(jobs) >= num_jobs:
                break

            try:
                job_card.click()
                time.sleep(2)
            except:
                continue

            # Safe text finder
            def safe_find(selector, by=By.CSS_SELECTOR):
                try:
                    return driver.find_element(by, selector).text
                except NoSuchElementException:
                    return -1

            # Safe attribute finder
            def safe_attr(selector, attr, by=By.CSS_SELECTOR):
                try:
                    return driver.find_element(by, selector).get_attribute(attr)
                except NoSuchElementException:
                    return -1

            job_title = safe_find("a[data-test='job-title']")
            job_url = safe_attr("a[data-test='job-title']", "href")
            company_name = safe_find("span.EmployerProfile_compactEmployerName__9MGcV")
            location = safe_find("div[data-test='emp-location']")
            job_description = safe_find("div.jobDescriptionContent")

            jobs.append({
                "Job Title": job_title,
                "Job Location": location,
                "Company Name": company_name,
                "Job Description": job_description,
                "Source URL": job_url
            })

        # Go to next page
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "button[data-test='pagination-next']")
            if next_button.is_enabled():
                next_button.click()
                time.sleep(slp_time)
            else:
                break
        except NoSuchElementException:
            break

    driver.quit()
    return pd.DataFrame(jobs)


# ---------------- Run scraper ----------------
df = get_jobs("data scientist", 20, True, 4)
df.to_csv("Uncleaned_DS_jobs.csv", index=False, encoding="utf-8")
print(f"✅ Got {len(df)} jobs, saved to Uncleaned_DS_jobs.csv")

✅ Got 20 jobs, saved to Uncleaned_DS_jobs.csv


In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time


def get_jobs(keyword, num_jobs, slp_time=3):
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.set_window_size(1200, 1000)

    url = f"https://www.glassdoor.com/Job/jobs.htm?sc.keyword={keyword}"
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:
        try:
            job_cards = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.JobCard_jobCardWrapper__vX29z"))
            )
        except TimeoutException:
            print("⚠️ No job cards found. Structure may have changed.")
            break

        for card in job_cards:
            if len(jobs) >= num_jobs:
                break

            try:
                title_elem = card.find_element(By.CSS_SELECTOR, "a[data-test='job-title']")
                job_title = title_elem.text
                job_url = title_elem.get_attribute("href")
            except:
                job_title, job_url = -1, -1

            try:
                company_name = card.find_element(By.CSS_SELECTOR, "span.EmployerProfile_compactEmployerName__9MGcV").text
            except:
                company_name = -1

            try:
                location = card.find_element(By.CSS_SELECTOR, "div[data-test='emp-location']").text
            except:
                location = -1

            # Wait for job description in side panel
            try:
                card.click()
                desc_elem = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.jobDescriptionContent"))
                )
                job_description = desc_elem.text
            except:
                job_description = -1

            jobs.append({
                "Job Title": job_title,
                "Job Location": location,
                "Company Name": company_name,
                "Job Description": job_description,
                "Source URL": job_url
            })

        # Try to go to next page
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "button[data-test='pagination-next']")
            if next_button.is_enabled():
                next_button.click()
                time.sleep(slp_time)
            else:
                break
        except NoSuchElementException:
            break

    driver.quit()
    return pd.DataFrame(jobs)


# ---------------- Run scraper ----------------
df = get_jobs("data scientist", 20, 3)
df.to_csv("Uncleaned_DS_jobs.csv", index=False, encoding="utf-8")
print(f"✅ Got {len(df)} jobs, saved to Uncleaned_DS_jobs.csv")

✅ Got 20 jobs, saved to Uncleaned_DS_jobs.csv


In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time


def get_jobs(keyword, num_jobs, slp_time=3):
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.set_window_size(1200, 1000)

    url = f"https://www.glassdoor.com/Job/jobs.htm?sc.keyword={keyword}"
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:
        try:
            job_cards = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.JobCard_jobCardWrapper__vX29z"))
            )
        except TimeoutException:
            print("⚠️ No job cards found. Structure may have changed.")
            break

        for card in job_cards:
            if len(jobs) >= num_jobs:
                break

            try:
                title_elem = card.find_element(By.CSS_SELECTOR, "a[data-test='job-title']")
                job_title = title_elem.text
                job_url = title_elem.get_attribute("href")
            except:
                job_title, job_url = -1, -1

            try:
                company_name = card.find_element(By.CSS_SELECTOR, "span.EmployerProfile_compactEmployerName__9MGcV").text
            except:
                company_name = -1

            try:
                location = card.find_element(By.CSS_SELECTOR, "div[data-test='emp-location']").text
            except:
                location = -1

            # Extract job description (side panel)
            try:
                card.click()
                desc_elem = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((
                        By.CSS_SELECTOR,
                        "div.JobDetails_jobDescription__uW_fK.JobDetails_showHidden__C_FOA"
                    ))
                )
                job_description = desc_elem.text.strip()
            except:
                # fallback if class changes
                try:
                    desc_elem = driver.find_element(By.CSS_SELECTOR, "div.jobDescriptionContent")
                    job_description = desc_elem.text.strip()
                except:
                    job_description = -1

            jobs.append({
                "Job Title": job_title,
                "Job Location": location,
                "Company Name": company_name,
                "Job Description": job_description,
                "Source URL": job_url
            })

        # Try to go to next page
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "button[data-test='pagination-next']")
            if next_button.is_enabled():
                next_button.click()
                time.sleep(slp_time)
            else:
                break
        except NoSuchElementException:
            break

    driver.quit()
    return pd.DataFrame(jobs)


# ---------------- Run scraper ----------------
df = get_jobs("data scientist", 20, 3)
df.to_csv("Uncleaned_DS_jobs.csv", index=False, encoding="utf-8")
print(f"✅ Got {len(df)} jobs, saved to Uncleaned_DS_jobs.csv")

✅ Got 20 jobs, saved to Uncleaned_DS_jobs.csv


In [31]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time


def get_jobs(keyword, num_jobs, slp_time=3):
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.set_window_size(1200, 1000)

    url = f"https://www.glassdoor.com/Job/jobs.htm?sc.keyword={keyword}"
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:
        try:
            job_cards = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.JobCard_jobCardWrapper__vX29z"))
            )
        except TimeoutException:
            print("⚠️ No job cards found. Structure may have changed.")
            break

        for card in job_cards:
            if len(jobs) >= num_jobs:
                break

            # --- Job Title & URL ---
            try:
                title_elem = card.find_element(By.CSS_SELECTOR, "a[data-test='job-title']")
                job_title = title_elem.text
                job_url = title_elem.get_attribute("href")
            except:
                job_title, job_url = -1, -1

            # --- Company Name ---
            try:
                company_name = card.find_element(By.CSS_SELECTOR, "span.EmployerProfile_compactEmployerName__9MGcV").text
            except:
                company_name = -1

            # --- Location ---
            try:
                location = card.find_element(By.CSS_SELECTOR, "div[data-test='emp-location']").text
            except:
                location = -1

            # --- Job Description (side panel) ---
            try:
                driver.execute_script("arguments[0].click();", card)

                # Wait until panel loads this job
                WebDriverWait(driver, 10).until(
                    EC.text_to_be_present_in_element((By.CSS_SELECTOR, "div[data-test='jobTitle']"), job_title)
                )

                # Try primary description selector
                desc_elem = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((
                        By.CSS_SELECTOR,
                        "div.JobDetails_jobDescription__uW_fK.JobDetails_showHidden__C_FOA"
                    ))
                )
                job_description = desc_elem.text.strip()

            except Exception:
                # Fallback: old selector
                try:
                    desc_elem = driver.find_element(By.CSS_SELECTOR, "div.jobDescriptionContent")
                    job_description = desc_elem.text.strip()
                except:
                    job_description = -1

            jobs.append({
                "Job Title": job_title,
                "Job Location": location,
                "Company Name": company_name,
                "Job Description": job_description,
                "Source URL": job_url
            })

        # --- Next page ---
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "button[data-test='pagination-next']")
            if next_button.is_enabled():
                next_button.click()
                time.sleep(slp_time)
            else:
                break
        except NoSuchElementException:
            break

    driver.quit()
    return pd.DataFrame(jobs)


# ---------------- Run scraper ----------------
df = get_jobs("data scientist", 20, 3)
df.to_csv("Uncleaned_DS_jobs.csv", index=False, encoding="utf-8")
print(f"✅ Got {len(df)} jobs, saved to Uncleaned_DS_jobs.csv")

✅ Got 20 jobs, saved to Uncleaned_DS_jobs.csv


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException
import csv

# Set up a controllable Chrome instance
driver = webdriver.Chrome(service=Service())

# Open the target page in the browser
driver.get("https://www.indeed.com/jobs?q=data+scientist&l=New+York%2C+NY&from=searchOnDesktopSerp")

# A data structure where to store the scraped job openings
jobs = []

# Select the job opening elements on the page
jobs_container_element = driver.find_element(By.CSS_SELECTOR, "#mosaic-provider-jobcards")
job_elements = jobs_container_element.find_elements(By.CSS_SELECTOR, "[data-testid="slider_item"]")

# Scrape each job opening on the page
for job_element in job_elements:
    title_element = job_element.find_element(By.CSS_SELECTOR, "h2.jobTitle")
    title = title_element.text

    url_element = title_element.find_element(By.CSS_SELECTOR, "a")
    url = url_element.get_attribute("href")

    company_element =job_element.find_element(By.CSS_SELECTOR, "[data-testid="company-name"]")
    company = company_element.text

    location_element = job_element.find_element(By.CSS_SELECTOR, "[data-testid="text-location"]")
    location = location_element.text

    tags = []
    tags_container_element = job_element.find_element(By.CSS_SELECTOR, ".jobMetaDataGroup")
    tag_elements = tags_container_element.find_elements(By.CSS_SELECTOR, "[data-testid="attribute_snippet_testid"]")
    for tag_element in tag_elements:
        tag = tag_element.text
        tags.append(tag)

    # Check whether the "Easy Apply" element is on the page
    try:
        job_element.find_element(By.CSS_SELECTOR, "[data-testid="indeedApply"]")
        easily_apply = True
    except NoSuchElementException:
        easily_apply = False

    description = []
    description_container_element = job_element.find_element(By.CSS_SELECTOR, "[role="presentation"]")
    description_elements = description_container_element.find_elements(By.CSS_SELECTOR, "ul li")
    for description_element in description_elements:
        description_item_text = description_element.text
        # Ignore empty description strings
        if (description_item_text != ""):
            description.append(description_item_text)

    # Store the scraped data
    job = {
        "title": title,
        "url": url,
        "company": company,
        "location": location,
        "tags": tags,
        "easily_apply": easily_apply,
        "description": description
    }
    jobs.append(job)

# Export the scraped data to an output CSV file
csv_file = "jobs.csv"
csv_headers = ["title", "url", "company", "location", "tags", "easily_apply", "description"]

with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=csv_headers)
    writer.writeheader()
    for job in jobs:
        writer.writerow({
            "title": job["title"],
            "url": job["url"],
            "company": job["company"],
            "location": job["location"],
            "tags": ";".join(job["tags"]),
            "easily_apply": "Yes" if job["easily_apply"] else "No",
            "description": ";".join(job["description"])
        })

# Close the web driver
driver.quit()

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException
import csv
import time
import urllib.parse


def get_jobs(job_title, num_jobs=20, num_pages=1, location="New York, NY"):
    """
    Scrape jobs from Indeed.

    job_title : str -> e.g. "data scientist"
    num_jobs  : int -> target number of jobs to scrape
    num_pages : int -> how many pages to scrape
    location  : str -> location string (default: New York, NY)
    """

    job_query = urllib.parse.quote_plus(job_title)
    loc_query = urllib.parse.quote_plus(location)

    driver = webdriver.Chrome(service=Service())
    jobs = []
    jobs_scraped = 0
    page = 0

    while jobs_scraped < num_jobs and page < num_pages:
        url = f"https://www.indeed.com/jobs?q={job_query}&l={loc_query}&start={page*10}"
        driver.get(url)
        time.sleep(2)

        try:
            jobs_container_element = driver.find_element(By.CSS_SELECTOR, "#mosaic-provider-jobcards")
            job_elements = jobs_container_element.find_elements(By.CSS_SELECTOR, "[data-testid='slider_item']")
        except Exception:
            print(f"No job container found on page {page+1}")
            break

        for job_element in job_elements:
            if jobs_scraped >= num_jobs:
                break

            try:
                # Title & URL
                title_element = job_element.find_element(By.CSS_SELECTOR, "h2.jobTitle a")
                title = title_element.text
                job_url = title_element.get_attribute("href")

                # Company
                company_element = job_element.find_element(By.CSS_SELECTOR, "[data-testid='company-name']")
                company = company_element.text

                # Location
                location_element = job_element.find_element(By.CSS_SELECTOR, "[data-testid='text-location']")
                job_location = location_element.text

                # Description
                description = []
                try:
                    desc_container = job_element.find_element(By.CSS_SELECTOR, "[role='presentation']")
                    desc_elements = desc_container.find_elements(By.CSS_SELECTOR, "ul li")
                    description = [d.text for d in desc_elements if d.text.strip() != ""]
                except NoSuchElementException:
                    pass

                job = {
                    "Job Title": title,
                    "Job Location": job_location,
                    "Company Name": company,
                    "Job Description": " ".join(description),
                    "Source URL": job_url
                }
                jobs.append(job)
                jobs_scraped += 1

            except Exception as e:
                print(f"Error scraping a job: {e}")
                continue

        page += 1

    # Save CSV
    csv_file = "jobs.csv"
    csv_headers = ["Job Title", "Job Location", "Company Name", "Job Description", "Source URL"]

    with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=csv_headers)
        writer.writeheader()
        for job in jobs:
            writer.writerow(job)

    driver.quit()
    return jobs


# ---------------- Run scraper ----------------
if __name__ == "__main__":
    jobs = get_jobs("data scientist", num_jobs=20, num_pages=3, location="New York, NY")
    print(f"Scraped {len(jobs)} jobs")